In [ ]:
import os
import pandas as pd
from sklearn.utils import shuffle
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from skimage import io
from sklearn.model_selection import train_test_split
import numpy as np
from torchsummary import summary

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d oluwaseunad/concrete-and-pavement-crack-images

100% 318M/319M [00:17<00:00, 13.9MB/s]
100% 319M/319M [00:17<00:00, 19.6MB/s]


In [ ]:
!unzip -q concrete-and-pavement-crack-images.zip

In [ ]:
!ls

concrete-and-pavement-crack-images.zip	kaggle.json  Negative  Positive  sample_data


In [ ]:
# Load data paths and labels
path_Negative = '/content/Negative'
path_Positive = '/content/Positive'
positive_images = [os.path.join(path_Positive, img) for img in os.listdir(path_Positive)]
negative_images = [os.path.join(path_Negative, img) for img in os.listdir(path_Negative)]
all_images = positive_images + negative_images
labels = [1] * len(positive_images) + [0] * len(negative_images)  # Convert to integers
df = pd.DataFrame(list(zip(all_images, labels)), columns=['Filepath', 'Label'])

In [ ]:
df = shuffle(df, random_state=42)

In [ ]:
df.head(5)

,Filepath,Label
2308,/content/Positive/14606.jpg,1
22404,/content/Negative/14409.jpg,0
23397,/content/Negative/05699.jpg,0
25058,/content/Negative/03792.jpg,0
2664,/content/Positive/02665.jpg,1


In [ ]:
df["Label"].value_counts()

1    15000
0    15000
Name: Label, dtype: int64

In [ ]:
# Data Preprocessing
class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, 0]
        image = io.imread(img_name)
        label = int(self.df.iloc[idx, 1])
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
])

In [ ]:
# from PIL import Image
# import torchvision.transforms as transforms
# import matplotlib.pyplot as plt

# # Define the transformation
# transform_ = transforms.Compose([
#     transforms.Resize((100, 100)),
#     transforms.ToTensor(),
# ])

# # Function to load and transform the image
# def load_and_transform_image(image_path, transform):
#     # Load the image
#     img = Image.open(image_path)
#     # Apply the transformation
#     transformed_img = transform(img)
#     return transformed_img

# image_path = "/content/Positive/00017.jpg"

# # Load and transform the image
# transformed_image = load_and_transform_image(image_path, transform_)

# # Convert tensor to numpy array and transpose the dimensions
# # to (height, width, channels) for visualization
# image_np = transformed_image.numpy().transpose((1, 2, 0))

# # Visualize the transformed image
# plt.imshow(image_np)
# plt.axis('off')
# plt.show()


In [ ]:
# Split data into train and test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_dataset = CustomDataset(train_df, transform=transform)
test_dataset = CustomDataset(test_df, transform=transform)

In [ ]:
trainloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# Define the model
class Rasnet50(nn.Module):
    def __init__(self):
        super(Rasnet50, self).__init__()
        self.resnet = torchvision.models.resnet50(pretrained=True)
        # Freeze ResNet parameters
        for param in self.resnet.parameters():
            param.requires_grad = False

        self.resnet.fc = nn.Sequential(
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        x = self.resnet(x)
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
rasnet50 = Rasnet50()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 55.5MB/s]


In [ ]:
rasnet50.to(device)

Rasnet50(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          

In [ ]:
summary(rasnet50,(3,100,100))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 50, 50]           9,408
       BatchNorm2d-2           [-1, 64, 50, 50]             128
              ReLU-3           [-1, 64, 50, 50]               0
         MaxPool2d-4           [-1, 64, 25, 25]               0
            Conv2d-5           [-1, 64, 25, 25]           4,096
       BatchNorm2d-6           [-1, 64, 25, 25]             128
              ReLU-7           [-1, 64, 25, 25]               0
            Conv2d-8           [-1, 64, 25, 25]          36,864
       BatchNorm2d-9           [-1, 64, 25, 25]             128
             ReLU-10           [-1, 64, 25, 25]               0
           Conv2d-11          [-1, 256, 25, 25]          16,384
      BatchNorm2d-12          [-1, 256, 25, 25]             512
           Conv2d-13          [-1, 256, 25, 25]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rasnet50.resnet.fc.parameters(), lr=0.001)

In [ ]:

correct = 0
total = 0
no_of_epoch = 10
for epoch in range(no_of_epoch):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = rasnet50(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:  # print every 10 mini-batches
            print(f"Epoch [{epoch + 1}/{no_of_epoch}], Batch [{i + 1}/{len(trainloader)}], Loss: {running_loss / 10:.3f} (epoch loss)")
            running_loss = 0.0

        # Calculate accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the {total} train images: %d %%' % (
        100 * correct / total))
    correct = 0
    total = 0

print('Finished Training')




Epoch [1/10], Batch [10/375], Loss: 0.557 (epoch loss)
Epoch [1/10], Batch [20/375], Loss: 0.263 (epoch loss)
Epoch [1/10], Batch [30/375], Loss: 0.176 (epoch loss)
Epoch [1/10], Batch [40/375], Loss: 0.120 (epoch loss)
Epoch [1/10], Batch [50/375], Loss: 0.103 (epoch loss)
Epoch [1/10], Batch [60/375], Loss: 0.103 (epoch loss)
Epoch [1/10], Batch [70/375], Loss: 0.226 (epoch loss)
Epoch [1/10], Batch [80/375], Loss: 0.173 (epoch loss)
Epoch [1/10], Batch [90/375], Loss: 0.127 (epoch loss)
Epoch [1/10], Batch [100/375], Loss: 0.142 (epoch loss)
Epoch [1/10], Batch [110/375], Loss: 0.151 (epoch loss)
Epoch [1/10], Batch [120/375], Loss: 0.163 (epoch loss)
Epoch [1/10], Batch [130/375], Loss: 0.228 (epoch loss)
Epoch [1/10], Batch [140/375], Loss: 0.115 (epoch loss)
Epoch [1/10], Batch [150/375], Loss: 0.125 (epoch loss)
Epoch [1/10], Batch [160/375], Loss: 0.105 (epoch loss)
Epoch [1/10], Batch [170/375], Loss: 0.107 (epoch loss)
Epoch [1/10], Batch [180/375], Loss: 0.068 (epoch loss)
E

In [ ]:
# Save entire model
torch.save(rasnet50, 'rasnet50_pytorch.pth')

In [ ]:
# Load entire model
model = torch.load('rasnet50_pytorch.pth')

In [ ]:
import torch
from sklearn.metrics import roc_auc_score, f1_score
from tqdm import tqdm

# Assuming testloader is your DataLoader for the test dataset

# Initialize variables to keep track of metrics
predictions = []
true_labels = []

# Switch model to evaluation mode
model.eval()

with torch.no_grad():  # No need to compute gradients during evaluation
    for data in tqdm(testloader, desc="Testing"):  # Loop over the test dataset
        inputs, labels = data[0].to(device), data[1].to(device)

        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = criterion(outputs, labels)

        # Store predictions and true labels for metric calculation
        output = outputs.cpu().numpy()
        outputs = np.argmax(output,axis=1)
        predictions.extend(outputs)

        # predictions.extend()
        # print(predictions[0])
        true_labels.extend(labels.cpu().numpy())  #sorted


# Calculate ROC AUC score
roc_auc = roc_auc_score(true_labels, predictions)

# Calculate F1 score
f1 = f1_score(true_labels, predicted_labels)

# Calculate accuracy
accuracy = (predicted_labels == true_labels).mean()

print(f"ROC AUC: {roc_auc:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Testing: 100%|██████████| 94/94 [00:18<00:00,  5.11it/s]

ROC AUC: 0.9887
F1 Score: 0.9891
Accuracy: 98.88%
